## BONUS : using pg_fasttransfer extension to run fasttransfer from psql command line


```
██████   ██████          ███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██   ██ ██               ██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
██████  ██   ███         █████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██    ██         ██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██       ██████  ███████ ██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 


Get the extension from : https://github.com/aetperf/pg_fasttransfer


### Installation of the extension on the source database (pg16)

- Download the extension for your system (windows or linux) and your postgresql version (16 or 17) on the release page
https://github.com/aetperf/pg_fasttransfer/releases/tag/v0.6.1
- unzip in a temp folder
- run the installer (bat or sh script) as the admin user (warning : it will install the extension in the postgresql server folders and then must restart the postgresql service)

More details in the README file of the github project.

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.PostgreSQL , *-*"

Installed Packages Microsoft.DotNet.Interactive.PostgreSQL, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.postgresql\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query PostgreSQL databases. 
 This extension adds support for connecting to PostgreSql databases using the #!connect postgres magic command.

In [2]:
// Connect to PostgreSQL fasttransfer database (that will have the pg_fasttransfer extension installed)
#!connect postgres "Host=localhost;Port=25432;Database=fasttransfer;Username=FastUser;Password=FastPassword;CommandTimeout=1500" --kernel-name pgsql17_fasttransfer

Kernel added: #!sql-pgsql17_fasttransfer

In [3]:
// Connect to PostgreSQL target database 
#!connect postgres "Host=localhost;Port=25432;Database=tpch;Username=FastUser;Password=FastPassword;CommandTimeout=1500" --kernel-name pgsql17_tpch

Kernel added: #!sql-pgsql17_tpch

In [5]:
DROP EXTENSION IF EXISTS pg_fasttransfer;
CREATE EXTENSION pg_fasttransfer CASCADE;
SELECT * FROM pg_available_extensions WHERE name='pg_fasttransfer';

name,default_version,installed_version,comment
pg_fasttransfer,1.0,1.0,PostgreSQL FastTransfer extension


In [6]:
TRUNCATE TABLE tpch_10.orders_heap;
SELECT count(*) FROM tpch_10.orders_heap;

count
0


### Run FastTransfer from psql command line using the xp_fasttransfer_secure "procedure"

In [13]:
SELECT * from xp_RunFastTransfer_secure(
sourceconnectiontype := 'pgsql',
sourceserver := 'localhost:15432',
sourceuser := 'FastUser',
sourcepassword := 'ww0EBwMCc/4pDzBaadRn0j0BCJu5ZTZ0h64a3Q/AB+DBLhsa8mBkasUdEhTsM6slUOLZ43pVPSBs
XjoB0/SMxNdh6jq3wVeBf3Iez1Wn',
sourcedatabase := 'tpch',
sourceschema := 'tpch_10',
sourcetable := 'orders',
targetconnectiontype := 'pgcopy',
targetserver := 'localhost:25432',
targetuser := 'FastUser',
targetpassword := 'ww0EBwMCc/4pDzBaadRn0j0BCJu5ZTZ0h64a3Q/AB+DBLhsa8mBkasUdEhTsM6slUOLZ43pVPSBs
XjoB0/SMxNdh6jq3wVeBf3Iez1Wn', 
targetdatabase := 'tpch',
targetschema := 'tpch_10',
targettable := 'orders_heap',
method := 'Ctid',
degree := 12,
loadmode := 'Truncate',
batchsize := 1048576,
mapmethod := 'Name',
fasttransfer_path := 'D:\FastTransfer'
);

exit_code,output,error_message,total_rows,total_columns,transfer_time_ms,total_time_ms
0,,,15000000,9,6184,6346


In [14]:
SELECT count(*) FROM tpch_10.orders_heap;

count
15000000


In [15]:
SELECT * FROM tpch_10.orders_heap ORDER BY o_orderkey LIMIT 10;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
1,369001,O,186600.000,1996-01-02 00:00:00Z,5-LOW,Clerk#000009506,0,ly express platelets. deposits acc
2,780017,O,66220.000,1996-12-01 00:00:00Z,1-URGENT,Clerk#000008792,0,ve the furiously fluffy dependencies. carefully regular
3,1233140,F,270742.000,1993-10-14 00:00:00Z,5-LOW,Clerk#000009543,0,after the asymptotes. instructions cajole after the foxes. carefully unu
4,1367761,O,41714.000,1995-10-11 00:00:00Z,5-LOW,Clerk#000001234,0,st the furiously bold pinto beans. furiously pending theodolites cajol
5,444848,F,122444.000,1994-07-30 00:00:00Z,5-LOW,Clerk#000009248,0,onic requests. carefully daring foxes among the carefu
6,556222,F,50884.000,1992-02-21 00:00:00Z,4-NOT SPECIFIED,Clerk#000000580,0,furiously ironic accounts haggle blithely carefully regular de
7,391343,O,287535.000,1996-01-10 00:00:00Z,2-HIGH,Clerk#000004697,0,", ironic packages wa"
32,1300570,O,129635.000,1995-07-16 00:00:00Z,2-HIGH,Clerk#000006157,0,"ly about the carefully express theodolites. ironic, iron"
33,669580,F,126999.000,1993-10-27 00:00:00Z,3-MEDIUM,Clerk#000004086,0,"careful, regular courts. unusual"
34,610001,O,55315.000,1998-07-21 00:00:00Z,3-MEDIUM,Clerk#000002228,0,osits according to the ideas are furiously final requests? slyly pe


### Source Oracle ORDERS Target PostgreSQL ORDERS table (with PK)

In [32]:
TRUNCATE TABLE tpch_10.orders;
SELECT count(*) FROM tpch_10.orders;

count
0


In [28]:
SELECT pg_fasttransfer_encrypt('TPCH_IN');

pg_fasttransfer_encrypt
ww0EBwMClLUgzaK7CeV/0jgBrcJ12PhQaPFsrHKtbT8h0vzJkZZah65ju5+jL9niJT/YAGYccAgz Et+WdkYpnW9Cp3RklNy1Gw==


In [16]:
SELECT * from xp_RunFastTransfer_secure(
sourceconnectiontype := 'oraodp',
sourceserver := 'localhost:1521/ORCLPDB',
sourceuser := 'TPCH_IN',
sourcepassword := 'ww0EBwMCdYjkD/4zFQZ60jgB7UXs/d3NLuwAt9AEa6byGxM/m8bliOVmug4+1gWggjaOtbecbqij 7BFiqov+h7slviPMgzWokA==',
sourcedatabase := 'ORCLPDB',
sourceschema := 'TPCH_IN',
sourcetable := 'ORDERS_FLAT',
targetconnectiontype := 'pgcopy',
targetserver := 'localhost:25432',
targetuser := 'FastUser',
targetpassword := 'ww0EBwMCc/4pDzBaadRn0j0BCJu5ZTZ0h64a3Q/AB+DBLhsa8mBkasUdEhTsM6slUOLZ43pVPSBs XjoB0/SMxNdh6jq3wVeBf3Iez1Wn', 
targetdatabase := 'tpch',
targetschema := 'tpch_10',
targettable := 'orders',
method := 'RangeId',
distributekeycolumn := 'O_ORDERKEY',
degree := 12,
loadmode := 'Truncate',
batchsize := 1048576,
mapmethod := 'Name',
fasttransfer_path := 'D:\FastTransfer'
);

exit_code,output,error_message,total_rows,total_columns,transfer_time_ms,total_time_ms
0,,,15000000,9,14567,15025


In [34]:
SELECT count(*) FROM tpch_10.orders;

count
15000000


### Source SQL Server ORDERS Target PostgreSQL ORDERS table (with PK)